In [ ]:
# Python imports and pre-definitions
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['font.size'] = 30

def parse_lammps_rdf(rdffile):
    """Parse the RDF file written by LAMMPS
    copied from Boris' class code: https://github.com/bkoz37/labutil
    """
    with open(rdffile, 'r') as rdfout:
        rdfs = []; buffer = []
        for line in rdfout:
            values = line.split()
            if line.startswith('#'):
                continue
            elif len(values) == 2:
                nbins = values[1]
            else:
                buffer.append([float(values[1]), float(values[2])])
                if len(buffer) == int(nbins):
                    frame = np.transpose(np.array(buffer))
                    rdfs.append(frame)
                    buffer = []
    return rdfs

In [ ]:
!nequip-train configs/tutorial.yaml --equivariance-test

In [ ]:
!nequip-benchmark configs/tutorial.yaml

In [ ]:
!nequip-evaluate --train-dir results/silicon-tutorial/si --batch-size 1

In [ ]:
!nequip-deploy build --train-dir results/silicon-tutorial/si si-deployed.pth

In [ ]:
from ase.io import read, write

example_atoms = read('./data/toluene.xyz', index=0)
write('./data/toluene.data', example_atoms, format='lammps-data')

In [ ]:
lammps_input = """
units	metal
atom_style atomic
dimension 3

# set newton on for pair_allegro (off for pair_nequip)
newton on
boundary p p p
read_data ../si.data

# if you want to run a larger system, simply replicate the system in space
# replicate 3 3 3

# allegro pair style
pair_style	allegro
pair_coeff	* * ../si-deployed.pth Si

mass 1 28.0855 

velocity all create 300.0 1234567 loop geom

neighbor 1.0 bin
neigh_modify delay 5 every 1

timestep 0.001
thermo 10

# nose-hoover thermostat, 1500K
fix  1 all nvt temp 1500 1500 $(100*dt)

# compute rdf and average after some equilibration
comm_modify cutoff 7.0
compute rdfall all rdf 1000 cutoff 5.0
fix 2 all ave/time 1 2500 5000 c_rdfall[*] file si.rdf mode vector

# run 25ps
run 25000
"""  
!rm -rf ./lammps_run  
!mkdir lammps_run
with open("lammps_run/si_rdf.in", "w") as f:
    f.write(lammps_input)

In [ ]:
### cmake for LAMMPS ###
# Go inside lammps/build folder
!cmake ../cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_PREFIX_PATH=`python -c 'import torch;print(torch.utils.cmake_prefix_path)'` -DMKL_INCLUDE_DIR="$CONDA_PREFIX/include" -DCUDA_TOOLKIT_ROOT_DIR="/apps/cuda-11.1"
!make -j$(nproc)

In [ ]:
!mpirun -np 4 ../../lammps/build/lmp -in si_rdf.in

In [ ]:
rdf = parse_lammps_rdf('./lammps_run/md22_Ac-Ala3-NHMe.rdf')  # utility function defined earlier
plt.figure(figsize=(15, 8))
plt.plot(rdf[0][0], rdf[0][1], 'b', linewidth=5, label="Allegro, $T=300K$")
plt.xlabel('r [$\AA$]')
plt.ylabel('g(r)')
plt.title("Si-Si bond length: {:.3f}$\AA$".format(rdf[0][0][np.argmax(rdf[0][1])]))
plt.legend(loc='upper right')
plt.show()    

In [ ]:
import torch

### COMPUTE CURL ###

v = torch.ones((10, 3))
v[:5, :2] += 1
v[7] += 3

w = torch.ones((10, 3))
w[:2, :2] -= 3
w[4] += 1

external_grads_list = []
for i in range(len(v)):
    external_grad = torch.zeros_like(v)
    external_grad[i, 0] = 1.
    external_grads_list.append(external_grad)
    external_grad = torch.zeros_like(v)
    external_grad[i, 1] = 1.
    external_grads_list.append(external_grad)
    external_grad = torch.zeros_like(v)
    external_grad[i, 2] = 1.
    external_grads_list.append(external_grad)

# v = v.reshape(1, 30)
v.requires_grad = True

out = v * w #model(v)

grads_list = []
triplet_list = []
for i, external_grad in enumerate(external_grads_list):
    grads = torch.autograd.grad(
                [out],
                [v],
                retain_graph=True,
                grad_outputs=external_grad,
            )[0][i//3]
    triplet_list.append(grads)
    if len(triplet_list) == 3:
        grads_list.append(triplet_list)
        triplet_list = []
gradients = torch.stack([torch.stack(grads) for grads in grads_list])
cx = gradients[:, 2, 1] - gradients[:, 1, 2]
cy = gradients[:, 0, 2] - gradients[:, 2, 0]
cz = gradients[:, 1, 0] - gradients[:, 0, 1]
curl = torch.stack([cx, cy, cz]).T